# Step 06: Chat with Multiple Plugins

This notebook demonstrates interactive chat using multiple plugins (Location and DateTime).

In [ ]:
import asyncio
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.contents.chat_history import ChatHistory
from plugins.datetime import DateTimePlugin
from plugins.location import LocationPlugin

In [ ]:
# Initialize kernel and plugins
kernel = Kernel()
azure_chat_completion = AzureChatCompletion()
kernel.add_service(azure_chat_completion)

kernel.add_plugin(LocationPlugin(), plugin_name="location_plugin")
kernel.add_plugin(DateTimePlugin(), plugin_name="time_plugin")

execution_settings = AzureChatPromptExecutionSettings()

In [ ]:
# Get location and time context
location_function = kernel.get_function(
    plugin_name="location_plugin", function_name="GetLocationInfo"
)
location_info = await kernel.invoke(location_function)

time = kernel.get_function(plugin_name="time_plugin", function_name="Time")
current_time = await kernel.invoke(time)

print(f"Location: {location_info}")
print(f"Time: {current_time}")

In [ ]:
# Initialize chat history with context
history = ChatHistory()
history.add_system_message("User location: " + str(location_info))
history.add_system_message("Time that User started this chat: " + str(current_time))

In [ ]:
# Interactive chat function
async def chat(user_input: str):
    history.add_user_message(user_input)
    
    result = await azure_chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )
    
    print("Assistant > " + str(result))
    history.add_message(result)
    return result

In [ ]:
# Example chat interaction
await chat("What's the weather like where I am?")